In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import joblib
import os
warnings.filterwarnings('ignore')

In [4]:
model_path = './'

In [5]:
source_lgb_b_cv_0 = joblib.load(model_path + 'lgb_b_cv_0.pkl')
source_lgb_b_cv = joblib.load(model_path + 'lgb_b_cv.pkl')

In [94]:
columns = ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform",
           "source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num","create_role_pay_cost",
           "create_role_pay_sum","create_role_roi","create_role_retain_1d"]

In [95]:
data=[[20143,228091,3,"2020-05-01","2019-11-06",10,1000954,2,0.00,0,0E-7,0,0.000000,0.00,0E-7,0.0000],
      [20755,228435,3,"2020-05-01","2019-11-07",10,1000954,2,0.00,0,0E-7,0,0.000000,0.00,0E-7,0.0000]]

In [96]:
src_source_info = pd.DataFrame(data=data,columns=columns)

In [97]:
src_source_info.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
0,20143,228091,3,2020-05-01,2019-11-06,10,1000954,2,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1,20755,228435,3,2020-05-01,2019-11-07,10,1000954,2,0.0,0,0.0,0,0.0,0.0,0.0,0.0


In [98]:
src_source_info_0 = src_source_info[src_source_info['data_win'] == 0]
src_source_info_1_7 = src_source_info[src_source_info['data_win'] != 0]

In [99]:
src_source_info_0.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d


In [100]:
columns = [
      "channel_id",
      "source_id",
      "data_win",
      "source_run_datetime",
      "create_time",
      "media_id",
      "game_id",
      "platform",
      "source_run_date_amount",
      "create_role_num",
      "create_role_cost",
      "create_role_pay_num",
      "create_role_pay_cost",
      "create_role_pay_sum",
      "create_role_roi",
      "create_role_retain_1d",
      "label"
    ]
source_predict = pd.DataFrame(columns=columns)

In [101]:
drop_features_0 = ["channel_id","source_id","source_run_datetime","create_time","media_id","game_id","create_role_pay_num","create_role_pay_sum","create_role_retain_1d"]
drop_features_1_7 = ["channel_id", "source_id", "source_run_datetime", "create_time", "media_id", "game_id","create_role_pay_num", "create_role_pay_sum"]

In [102]:
if src_source_info_0.shape[0] != 0:
    src_source_info_0['label'] = source_lgb_b_cv_0.predict(src_source_info_0.drop(drop_features_0, axis=1))
    src_source_info_0['label'] = src_source_info_0['label'].apply(lambda x: 1 if x >= 0.7 else 0)
if src_source_info_1_7.shape[0] != 0:
    src_source_info_1_7['label'] = source_lgb_b_cv.predict(src_source_info_1_7.drop(drop_features_1_7, axis=1))
    src_source_info_1_7['label'] = src_source_info_1_7['label'].apply(lambda x: 1 if x >= 0.5 else 0)

In [103]:
source_predict = source_predict.append(src_source_info_0).append(src_source_info_1_7)

In [106]:
source_predict['label'] = source_predict['label'].astype(int)

In [65]:
# 数据输出
if source_predict.shape[0] == 0:
    source_predict.append(src_source_info_0)
    source_predict = pd.concat([src_source_info_0, src_source_info_1_7], axis=0)
else:
    source_predict = pd.concat([source_predict, src_source_info_0, src_source_info_1_7], axis=0)

In [107]:
source_predict

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,label
0,20143,228091,3,2020-05-01,2019-11-06,10,1000954,2,0.0,0,0.0,0,0.0,0.0,0.0,0.0,1
1,20755,228435,3,2020-05-01,2019-11-07,10,1000954,2,0.0,0,0.0,0,0.0,0.0,0.0,0.0,1


In [108]:
source_predict.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   channel_id              2 non-null      object 
 1   source_id               2 non-null      object 
 2   data_win                2 non-null      object 
 3   source_run_datetime     2 non-null      object 
 4   create_time             2 non-null      object 
 5   media_id                2 non-null      object 
 6   game_id                 2 non-null      object 
 7   platform                2 non-null      object 
 8   source_run_date_amount  2 non-null      float64
 9   create_role_num         2 non-null      object 
 10  create_role_cost        2 non-null      float64
 11  create_role_pay_num     2 non-null      object 
 12  create_role_pay_cost    2 non-null      float64
 13  create_role_pay_sum     2 non-null      float64
 14  create_role_roi         2 non-null      float6

In [22]:
source_predict.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,game_id,platform,amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,label
0,7162,299591,0,2020-05-01,2020-04-24,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.001590,0.0522,1
2,20754,304584,0,2020-05-01,2020-05-01,1001049,1,6603.77,640.0,10.334538,3.0,2201.256667,66.0,0.009994,0.0516,1
3,20754,304143,0,2020-05-01,2020-04-30,1000840,1,6640.08,406.0,16.476625,2.0,3320.040000,218.0,0.032831,0.0571,0
4,20754,301128,0,2020-05-01,2020-04-27,1000840,1,3850.09,270.0,14.474023,3.0,1283.363333,66.0,0.017143,0.0451,1
5,6770,301064,0,2020-05-01,2020-04-27,1001049,1,1621.59,256.0,6.409447,0.0,0.000000,0.0,0.000000,0.0593,1


In [44]:
source_predict.iloc[0:2]

,channel_id,source_id,data_win,source_run_datetime,create_time,game_id,platform,amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,label
0,7162,299591,0,2020-05-01,2020-04-24,1001049,1,200000.0,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522,1


In [15]:
# select_features_0 = ["data_win","platform","source_run_date_amount","create_role_num","create_role_cost","create_role_pay_cost","create_role_roi"]
drop_features = ["channel_id","source_id","source_run_datetime","create_time","game_id","create_role_pay_num","create_role_pay_sum","create_role_retain_1d"]
src_source_info_0['label'] = source_lgb_b_cv_0.predict(src_source_info_0.drop(drop_features, axis=1))
src_source_info_0['label'] = src_source_info_0['label'].apply(lambda x:1 if x>=0.7 else 0)

In [16]:
src_source_info_0.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,game_id,platform,amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,label
0,7162,299591,0,2020-05-01,2020-04-24,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.001590,0.0522,1
2,20754,304584,0,2020-05-01,2020-05-01,1001049,1,6603.77,640.0,10.334538,3.0,2201.256667,66.0,0.009994,0.0516,1
3,20754,304143,0,2020-05-01,2020-04-30,1000840,1,6640.08,406.0,16.476625,2.0,3320.040000,218.0,0.032831,0.0571,0
4,20754,301128,0,2020-05-01,2020-04-27,1000840,1,3850.09,270.0,14.474023,3.0,1283.363333,66.0,0.017143,0.0451,1
5,6770,301064,0,2020-05-01,2020-04-27,1001049,1,1621.59,256.0,6.409447,0.0,0.000000,0.0,0.000000,0.0593,1
